# Desafío 1 - Solución

## Paola Cartalá

### Consigna desafio 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

### Obtención y procesamiento de textos

In [1]:
# %pip install PyPDF2 numpy scikit-learn pandas tqdm

In [2]:
import re
import os
import glob
import json
import PyPDF2
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV

In [3]:
def extraer_texto_pdf(ruta_pdf):
    """
    Extrae texto de un archivo PDF.
    """
    try:
        with open(ruta_pdf, 'rb') as archivo:
            lector_pdf = PyPDF2.PdfReader(archivo)
            texto_completo = ""
            
            print(f"Procesando: {os.path.basename(ruta_pdf)} ({len(lector_pdf.pages)} páginas)")
            
            for pagina in tqdm(lector_pdf.pages, desc="Extrayendo páginas"):
                try:
                    texto_pagina = pagina.extract_text()
                    if texto_pagina:
                        texto_completo += texto_pagina + "\n"
                except Exception as e:
                    print(f"Error en página: {e}")
                    continue
            
            return texto_completo
    
    except Exception as e:
        print(f"Error procesando {ruta_pdf}: {e}")
        return None

In [4]:
def procesar_libros(carpeta_data="data"):
    """
    Procesa todos los libros de la carpeta data.
    """
    libros = []
    
    patron_pdf = os.path.join(carpeta_data, "*.pdf")
    archivos_pdf = glob.glob(patron_pdf)
    
    if not archivos_pdf:
        print(f"No se encontraron archivos PDF en la carpeta {carpeta_data}")
        return []
    
    print(f"Encontrados {len(archivos_pdf)} libros:")
    for archivo in archivos_pdf:
        print(f"  - {os.path.basename(archivo)}")
    
    for archivo_pdf in archivos_pdf:
        nombre_libro = os.path.splitext(os.path.basename(archivo_pdf))[0]
        
        print(f"\n{'='*60}")
        print(f"Procesando: {nombre_libro}")
        print(f"{'='*60}")
        
        texto = extraer_texto_pdf(archivo_pdf)
        
        if texto and len(texto.strip()) > 100:
            libros.append({
                'titulo': nombre_libro,
                'archivo': archivo_pdf,
                'texto': texto,
                'num_caracteres': len(texto),
                'num_palabras': len(texto.split())
            })
            print(f"✓ Texto extraído: {len(texto):,} caracteres, {len(texto.split()):,} palabras")
        else:
            print(f"✗ No se pudo extraer texto del archivo")
    
    print(f"\nProcesamiento completado. {len(libros)} libros procesados exitosamente.")
    return libros

In [5]:
libros_stephen_king = procesar_libros()
print(f"\nSe procesaron {len(libros_stephen_king)} libros de Stephen King")

Encontrados 5 libros:
  - El visitante - Stephen King.pdf
  - Fin de guardia - Stephen King.pdf
  - Holly - Stephen King.pdf
  - Mr Mercedes - Stephen King.pdf
  - Quien pierde paga - Stephen King.pdf

Procesando: El visitante - Stephen King
Procesando: El visitante - Stephen King.pdf (478 páginas)


Extrayendo páginas: 100%|██████████| 478/478 [00:27<00:00, 17.24it/s]



✓ Texto extraído: 1,000,020 caracteres, 175,368 palabras

Procesando: Fin de guardia - Stephen King
Procesando: Fin de guardia - Stephen King.pdf (291 páginas)


Extrayendo páginas: 100%|██████████| 291/291 [00:16<00:00, 18.17it/s]



✓ Texto extraído: 731,770 caracteres, 128,323 palabras

Procesando: Holly - Stephen King
Procesando: Holly - Stephen King.pdf (431 páginas)


...

✓ Texto extraído: 882,615 caracteres, 157,563 palabras

Procesando: Mr Mercedes - Stephen King
Procesando: Mr Mercedes - Stephen King.pdf (345 páginas)


Extrayendo páginas: 100%|██████████| 345/345 [00:23<00:00, 14.44it/s]



✓ Texto extraído: 851,392 caracteres, 150,535 palabras

Procesando: Quien pierde paga - Stephen King
Procesando: Quien pierde paga - Stephen King.pdf (349 páginas)


Extrayendo páginas: 100%|██████████| 349/349 [00:23<00:00, 15.01it/s]

✓ Texto extraído: 784,435 caracteres, 138,061 palabras

Procesamiento completado. 5 libros procesados exitosamente.

Se procesaron 5 libros de Stephen King


In [6]:
libros_metadata = []
for libro in libros_stephen_king:
    libros_metadata.append({
        'titulo': libro['titulo'],
        'archivo': libro['archivo'],
        'num_caracteres': libro['num_caracteres'],
        'num_palabras': libro['num_palabras']
    })

with open('stephen_king_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(libros_metadata, f, ensure_ascii=False, indent=2)
print("Metadatos guardados en stephen_king_metadata.json")

Metadatos guardados en stephen_king_metadata.json


In [7]:
def dividir_en_capitulos(texto, titulo_libro, tamaño_min_capitulo=2000):
    """
    Divide un libro en capítulos o secciones basándose en patrones comunes.
    """
    patrones_capitulo = [
        r'\n\s*(?:CAPÍTULO|CAPITULO|Chapter)\s+[IVXLCDM\d]+[^\n]*\n',
        r'\n\s*\d+\s*\n\n',
        r'\n\s*[IVXLCDM]+\s*\n\n',
        r'\n\s*(?:PARTE|PART)\s+[IVXLCDM\d]+[^\n]*\n'
    ]
    
    capitulos = []
    texto_restante = texto
    
    for patron in patrones_capitulo:
        divisiones = re.split(patron, texto_restante, flags=re.IGNORECASE)
        if len(divisiones) > 2:  # Si encontró divisiones útiles
            for i, division in enumerate(divisiones):
                if len(division.strip()) > tamaño_min_capitulo:
                    capitulos.append({
                        'libro': titulo_libro,
                        'capitulo': i + 1,
                        'texto': division.strip()
                    })
            break
    
    if not capitulos:
        palabras = texto.split()
        tamaño_chunk = 3000
        
        for i in range(0, len(palabras), tamaño_chunk):
            chunk = ' '.join(palabras[i:i + tamaño_chunk])
            if len(chunk.strip()) > tamaño_min_capitulo:
                capitulos.append({
                    'libro': titulo_libro,
                    'capitulo': i // tamaño_chunk + 1,
                    'texto': chunk.strip()
                })
    
    return capitulos

todos_los_documentos = []

for libro in libros_stephen_king:
    capitulos = dividir_en_capitulos(libro['texto'], libro['titulo'])
    todos_los_documentos.extend(capitulos)

print(f"\nTotal de documentos (capítulos) para análisis: {len(todos_los_documentos)}")


Total de documentos (capítulos) para análisis: 36


#### Preparación de datos para el análisis de texto

In [8]:
def preparar_textos(documentos):
    """
    Prepara los textos de los capítulos para análisis de NLP.
    """
    textos = []
    metadatos = []
    
    for doc in documentos:
        texto_limpio = re.sub(r'\s+', ' ', doc['texto']).strip()
        
        if len(texto_limpio) > 100:
            textos.append(texto_limpio)
            metadatos.append({
                'libro': doc['libro'],
                'capitulo': doc['capitulo'],
                'num_palabras': len(texto_limpio.split())
            })
    
    return textos, metadatos

textos_stephen_king, metadatos_stephen_king = preparar_textos(todos_los_documentos)

print(f"Se prepararon {len(textos_stephen_king)} textos para análisis")

Se prepararon 36 textos para análisis


### 1: Vectorización y análisis de similaridad de documentos

In [9]:
tfidfvect = TfidfVectorizer(
    max_features=5000,
    min_df=2,
    max_df=0.8,
    stop_words=None,
    lowercase=True,
    ngram_range=(1, 2)
)

X_stephen_king = tfidfvect.fit_transform(textos_stephen_king)

print(f"Matriz documento-término creada:")
print(f"Forma: {X_stephen_king.shape}")
print(f"Cantidad de documentos (capítulos): {X_stephen_king.shape[0]}")
print(f"Tamaño del vocabulario: {X_stephen_king.shape[1]}")
print(f"Densidad de la matriz: {X_stephen_king.nnz / (X_stephen_king.shape[0] * X_stephen_king.shape[1]):.4f}")

print(f"\nEjemplos de términos en el vocabulario:")
vocabulario_ejemplo = list(tfidfvect.vocabulary_.keys())[:20]
print(vocabulario_ejemplo)

Matriz documento-término creada:
Forma: (36, 5000)
Cantidad de documentos (capítulos): 36
Tamaño del vocabulario: 5000
Densidad de la matriz: 0.4904

Ejemplos de términos en el vocabulario:
['once', 'asesinado', 'pruebas', 'flint', 'city', 'terry', 'maitland', 'entrenador', 'liga', 'infantil', 'profesor', 'literatura', 'marido', 'ejemplar', 'niñas', 'detective', 'ralph', 'anderson', 'detención', 'firme']


In [10]:
np.random.seed(42)
indices_seleccionados = np.random.choice(len(textos_stephen_king), 5, replace=False)

print("Documentos (capítulos) seleccionados para análisis de similaridad:")
for i, idx in enumerate(indices_seleccionados):
    metadata = metadatos_stephen_king[idx]
    print(f"\n{i+1}. Documento {idx}")
    print(f"   Libro: {metadata['libro']}")
    print(f"   Capítulo: {metadata['capitulo']}")
    print(f"   Palabras: {metadata['num_palabras']}")
    print(f"   Texto (primeros 150 chars): {textos_stephen_king[idx][:150]}...")

Documentos (capítulos) seleccionados para análisis de similaridad:

1. Documento 35
   Libro: Quien pierde paga - Stephen King
   Capítulo: 5
   Palabras: 42190
   Texto (primeros 150 chars): El Padrino , pero es también una buena frase. Una de las mejores. Envía el dinero. Se queda el cuaderno. Un cuaderno caro que metió bajo la almohada c...

2. Documento 13
   Libro: Fin de guardia - Stephen King
   Capítulo: 4
   Palabras: 22203
   Texto (primeros 150 chars): —Ajá. Me ha dicho que empezó a notar a la señora Ellerton retraída… —Un poco retraída —rectifica la señora Alderson al instante—. En general, era la d...

3. Documento 26
   Libro: Mr Mercedes - Stephen King
   Capítulo: 4
   Palabras: 8587
   Texto (primeros 150 chars): 17 Ese primer interrogatorio crucial, solo unas horas después del crimen. Café y pastas mientras los cuerpos destrozados de los muertos eran identific...

4. Documento 30
   Libro: Mr Mercedes - Stephen King
   Capítulo: 8
   Palabras: 54226
   Texto (primeros

In [11]:
def analizar_similaridad_documento(idx, X, textos, metadatos, top_k=5):
    """
    Analiza la similaridad de un capítulo con todos los demás capítulos.
    """
    similaridades = cosine_similarity(X[idx], X)[0]
    
    indices_similares = np.argsort(similaridades)[::-1][1:top_k+1]
    
    print(f"\n{'='*70}")
    print(f"ANÁLISIS DE SIMILARIDAD - DOCUMENTO {idx}")
    print(f"{'='*70}")
    
    metadata_orig = metadatos[idx]
    print(f"\nDOCUMENTO ORIGINAL:")
    print(f"Libro: {metadata_orig['libro']}")
    print(f"Capítulo: {metadata_orig['capitulo']}")
    print(f"Palabras: {metadata_orig['num_palabras']}")
    print(f"Texto: {textos[idx][:200]}...")
    
    print(f"\nTOP {top_k} DOCUMENTOS MÁS SIMILARES:")
    for i, idx_sim in enumerate(indices_similares):
        metadata_sim = metadatos[idx_sim]
        sim_score = similaridades[idx_sim]
        
        mismo_libro = metadata_sim['libro'] == metadata_orig['libro']
        indicador = "📖 MISMO LIBRO" if mismo_libro else "📚 OTRO LIBRO"
        
        print(f"\n{i+1}. Similaridad: {sim_score:.4f} {indicador}")
        print(f"   Libro: {metadata_sim['libro']}")
        print(f"   Capítulo: {metadata_sim['capitulo']}")
        print(f"   Texto: {textos[idx_sim][:150]}...")
    
    return indices_similares, similaridades[indices_similares]

for idx in indices_seleccionados:
    analizar_similaridad_documento(idx, X_stephen_king, textos_stephen_king, metadatos_stephen_king)


ANÁLISIS DE SIMILARIDAD - DOCUMENTO 35

DOCUMENTO ORIGINAL:
Libro: Quien pierde paga - Stephen King
Capítulo: 5
Palabras: 42190
Texto: El Padrino , pero es también una buena frase. Una de las mejores. Envía el dinero. Se queda el cuaderno. Un cuaderno caro que metió bajo la almohada cuando la hermana menor apareció de improviso en la...

TOP 5 DOCUMENTOS MÁS SIMILARES:

1. Similaridad: 0.6663 📖 MISMO LIBRO
   Libro: Quien pierde paga - Stephen King
   Capítulo: 4
   Texto: atención. Me contrataron para encontrar el avión y tomar posesión de él. Eso es todo, punto y final. No trabajo para el FBI ni para el Departamento de...

2. Similaridad: 0.5809 📖 MISMO LIBRO
   Libro: Quien pierde paga - Stephen King
   Capítulo: 2
   Texto: candado enorme. Agarró de nuevo el asa, y esta vez se partió. —¡La puta! —exclamó Pete, y se miró las manos. Las tenía rojas y palpitantes. Bueno, de ...

3. Similaridad: 0.5004 📚 OTRO LIBRO
   Libro: Mr Mercedes - Stephen King
   Capítulo: 8
   Texto: llegado 

### 2: Optimización de modelos Naïve Bayes

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
def preparar_datos_clasificacion_libros(textos, metadatos, min_capitulos_por_libro=3):
    """
    Prepara los datos para clasificación por libro.
    """
    conteo_libros = {}
    for metadata in metadatos:
        libro = metadata['libro']
        conteo_libros[libro] = conteo_libros.get(libro, 0) + 1
    
    libros_validos = {libro for libro, count in conteo_libros.items() 
                     if count >= min_capitulos_por_libro}
    
    textos_filtrados = []
    etiquetas = []
    metadatos_filtrados = []
    
    for i, metadata in enumerate(metadatos):
        if metadata['libro'] in libros_validos:
            textos_filtrados.append(textos[i])
            etiquetas.append(metadata['libro'])
            metadatos_filtrados.append(metadata)
    
    return textos_filtrados, etiquetas, metadatos_filtrados, libros_validos

textos_clf, etiquetas_clf, metadatos_clf, libros_validos = preparar_datos_clasificacion_libros(
    textos_stephen_king, metadatos_stephen_king, min_capitulos_por_libro=3
)

#### Análisis de etiquetado temático y de personajes

Vamos a enriquecer la clasificación identificando temas y personajes principales en los textos.

In [ ]:
def detectar_temas_personajes(textos, metadatos):
    """
    Detecta temas y personajes principales en los textos basándose en palabras clave.
    """
    # Definir diccionarios de temas y personajes conocidos de Stephen King
    temas_keywords = {
        'horror_supernatural': ['fantasma', 'demonio', 'espíritu', 'supernatural', 'maldición', 'aparición', 'sobrenatural'],
        'crimen_detective': ['detective', 'policía', 'asesino', 'crimen', 'investigación', 'homicidio', 'caso'],
        'psicológico': ['mente', 'loco', 'psicólogo', 'mental', 'locura', 'cerebro', 'pensamiento'],
        'terror_urbano': ['ciudad', 'calle', 'hospital', 'hotel', 'edificio', 'urbano'],
        'muerte_violencia': ['muerte', 'sangre', 'matar', 'violencia', 'cadáver', 'asesinato']
    }
    
    personajes_keywords = {
        'bill_hodges': ['hodges', 'bill', 'detective hodges', 'william hodges'],
        'holly_gibney': ['holly', 'gibney', 'holly gibney'],
        'jerome_robinson': ['jerome', 'robinson'],
        'brady_hartsfield': ['brady', 'hartsfield', 'mr mercedes'],
        'ralph_anderson': ['ralph', 'anderson'],
        'el_visitante': ['visitante', 'outsider', 'extraño']
    }
    
    textos_etiquetados = []
    
    for i, texto in enumerate(textos):
        texto_lower = texto.lower()
        metadata = metadatos[i].copy()
        
        # Detectar temas
        temas_detectados = []
        for tema, keywords in temas_keywords.items():
            score = sum(1 for keyword in keywords if keyword in texto_lower)
            if score > 0:
                temas_detectados.append((tema, score))
        
        # Detectar personajes
        personajes_detectados = []
        for personaje, keywords in personajes_keywords.items():
            score = sum(1 for keyword in keywords if keyword in texto_lower)
            if score > 0:
                personajes_detectados.append((personaje, score))
        
        # Ordenar por relevancia
        temas_detectados.sort(key=lambda x: x[1], reverse=True)
        personajes_detectados.sort(key=lambda x: x[1], reverse=True)
        
        # Agregar etiquetas al metadata
        metadata['tema_principal'] = temas_detectados[0][0] if temas_detectados else 'general'
        metadata['todos_temas'] = [t[0] for t in temas_detectados]
        metadata['personaje_principal'] = personajes_detectados[0][0] if personajes_detectados else 'otros'
        metadata['todos_personajes'] = [p[0] for p in personajes_detectados]
        
        textos_etiquetados.append(metadata)
    
    return textos_etiquetados

# Aplicar etiquetado temático
metadatos_enriquecidos = detectar_temas_personajes(textos_stephen_king, metadatos_stephen_king)

print("Análisis de etiquetado temático y de personajes completado")
print(f"Ejemplo de metadatos enriquecidos:")
for i in range(min(3, len(metadatos_enriquecidos))):
    meta = metadatos_enriquecidos[i]
    print(f"\nCapítulo {i+1}:")
    print(f"  Libro: {meta['libro']}")
    print(f"  Tema principal: {meta['tema_principal']}")
    print(f"  Personaje principal: {meta['personaje_principal']}")
    print(f"  Todos los temas: {meta['todos_temas']}")
    print(f"  Todos los personajes: {meta['todos_personajes']}")

In [ ]:
def analizar_distribucion_etiquetas(metadatos_enriquecidos):
    """
    Analiza la distribución de temas y personajes en el corpus.
    """
    from collections import Counter
    
    # Contar temas principales
    temas_principales = [meta['tema_principal'] for meta in metadatos_enriquecidos]
    contador_temas = Counter(temas_principales)
    
    # Contar personajes principales
    personajes_principales = [meta['personaje_principal'] for meta in metadatos_enriquecidos]
    contador_personajes = Counter(personajes_principales)
    
    # Contar todos los temas (múltiple por documento)
    todos_temas = []
    for meta in metadatos_enriquecidos:
        todos_temas.extend(meta['todos_temas'])
    contador_todos_temas = Counter(todos_temas)
    
    # Contar todos los personajes (múltiple por documento)
    todos_personajes = []
    for meta in metadatos_enriquecidos:
        todos_personajes.extend(meta['todos_personajes'])
    contador_todos_personajes = Counter(todos_personajes)
    
    print("="*60)
    print("DISTRIBUCIÓN DE TEMAS Y PERSONAJES")
    print("="*60)
    
    print(f"\nTEMAS PRINCIPALES (uno por capítulo):")
    for tema, count in contador_temas.most_common():
        porcentaje = (count / len(metadatos_enriquecidos)) * 100
        print(f"  {tema}: {count} capítulos ({porcentaje:.1f}%)")
    
    print(f"\nPERSONAJES PRINCIPALES (uno por capítulo):")
    for personaje, count in contador_personajes.most_common():
        porcentaje = (count / len(metadatos_enriquecidos)) * 100
        print(f"  {personaje}: {count} capítulos ({porcentaje:.1f}%)")
    
    print(f"\nTODOS LOS TEMAS DETECTADOS (múltiples por capítulo):")
    for tema, count in contador_todos_temas.most_common():
        print(f"  {tema}: {count} menciones")
    
    print(f"\nTODOS LOS PERSONAJES DETECTADOS (múltiples por capítulo):")
    for personaje, count in contador_todos_personajes.most_common():
        print(f"  {personaje}: {count} menciones")
    
    return contador_temas, contador_personajes

# Analizar distribución
dist_temas, dist_personajes = analizar_distribucion_etiquetas(metadatos_enriquecidos)

In [ ]:
def entrenar_clasificadores_enriquecidos(textos, metadatos_enriquecidos):
    """
    Entrena clasificadores para temas y personajes además de libros.
    """
    print("\n" + "="*70)
    print("CLASIFICACIÓN ENRIQUECIDA: TEMAS Y PERSONAJES")
    print("="*70)
    
    # Preparar datos para clasificación por tema
    temas = [meta['tema_principal'] for meta in metadatos_enriquecidos]
    temas_unicos = list(set(temas))
    
    if len(temas_unicos) >= 2:
        print(f"\nClasificación por TEMA:")
        print(f"Temas disponibles: {temas_unicos}")
        
        # Filtrar solo temas con suficientes ejemplos
        from collections import Counter
        contador_temas = Counter(temas)
        temas_validos = [tema for tema, count in contador_temas.items() if count >= 3]
        
        if len(temas_validos) >= 2:
            textos_tema = []
            etiquetas_tema = []
            
            for i, meta in enumerate(metadatos_enriquecidos):
                if meta['tema_principal'] in temas_validos:
                    textos_tema.append(textos[i])
                    etiquetas_tema.append(meta['tema_principal'])
            
            if len(textos_tema) >= 10:
                # Entrenar clasificador de temas
                label_encoder_tema = LabelEncoder()
                y_tema_encoded = label_encoder_tema.fit_transform(etiquetas_tema)
                
                X_train_tema, X_test_tema, y_train_tema, y_test_tema = train_test_split(
                    textos_tema, y_tema_encoded, test_size=0.3, random_state=42, stratify=y_tema_encoded
                )
                
                # Vectorizar y entrenar
                vectorizer_tema = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
                X_train_tema_vec = vectorizer_tema.fit_transform(X_train_tema)
                X_test_tema_vec = vectorizer_tema.transform(X_test_tema)
                
                # Probar modelos
                modelo_tema = MultinomialNB(alpha=1.0)
                modelo_tema.fit(X_train_tema_vec, y_train_tema)
                
                y_pred_tema = modelo_tema.predict(X_test_tema_vec)
                f1_tema = f1_score(y_test_tema, y_pred_tema, average='macro')
                
                print(f"F1-score para clasificación de temas: {f1_tema:.4f}")
                print("Reporte de clasificación de temas:")
                print(classification_report(y_test_tema, y_pred_tema, 
                                          target_names=label_encoder_tema.classes_))
        else:
            print("No hay suficientes temas con ejemplos mínimos para entrenar.")
    
    # Preparar datos para clasificación por personaje
    personajes = [meta['personaje_principal'] for meta in metadatos_enriquecidos]
    personajes_unicos = list(set(personajes))
    
    if len(personajes_unicos) >= 2:
        print(f"\nClasificación por PERSONAJE:")
        print(f"Personajes disponibles: {personajes_unicos}")
        
        # Filtrar solo personajes con suficientes ejemplos
        contador_personajes = Counter(personajes)
        personajes_validos = [personaje for personaje, count in contador_personajes.items() if count >= 3]
        
        if len(personajes_validos) >= 2:
            textos_personaje = []
            etiquetas_personaje = []
            
            for i, meta in enumerate(metadatos_enriquecidos):
                if meta['personaje_principal'] in personajes_validos:
                    textos_personaje.append(textos[i])
                    etiquetas_personaje.append(meta['personaje_principal'])
            
            if len(textos_personaje) >= 10:
                # Entrenar clasificador de personajes
                label_encoder_personaje = LabelEncoder()
                y_personaje_encoded = label_encoder_personaje.fit_transform(etiquetas_personaje)
                
                X_train_pers, X_test_pers, y_train_pers, y_test_pers = train_test_split(
                    textos_personaje, y_personaje_encoded, test_size=0.3, random_state=42, stratify=y_personaje_encoded
                )
                
                # Vectorizar y entrenar
                vectorizer_personaje = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
                X_train_pers_vec = vectorizer_personaje.fit_transform(X_train_pers)
                X_test_pers_vec = vectorizer_personaje.transform(X_test_pers)
                
                # Probar modelos
                modelo_personaje = MultinomialNB(alpha=1.0)
                modelo_personaje.fit(X_train_pers_vec, y_train_pers)
                
                y_pred_personaje = modelo_personaje.predict(X_test_pers_vec)
                f1_personaje = f1_score(y_test_pers, y_pred_personaje, average='macro')
                
                print(f"F1-score para clasificación de personajes: {f1_personaje:.4f}")
                print("Reporte de clasificación de personajes:")
                print(classification_report(y_test_pers, y_pred_personaje, 
                                          target_names=label_encoder_personaje.classes_))
        else:
            print("No hay suficientes personajes con ejemplos mínimos para entrenar.")

# Entrenar clasificadores enriquecidos
entrenar_clasificadores_enriquecidos(textos_stephen_king, metadatos_enriquecidos)

In [ ]:
def analizar_similaridad_enriquecida(indices_seleccionados, X, textos, metadatos_enriquecidos, top_k=5):
    """
    Analiza la similaridad considerando las etiquetas temáticas y de personajes.
    """
    print("\n" + "="*70)
    print("ANÁLISIS DE SIMILARIDAD ENRIQUECIDO")
    print("="*70)
    
    for idx in indices_seleccionados:
        similaridades = cosine_similarity(X[idx], X)[0]
        indices_similares = np.argsort(similaridades)[::-1][1:top_k+1]
        
        print(f"\n{'='*60}")
        print(f"DOCUMENTO {idx} - ANÁLISIS ENRIQUECIDO")
        print(f"{'='*60}")
        
        metadata_orig = metadatos_enriquecidos[idx]
        print(f"\nDOCUMENTO ORIGINAL:")
        print(f"Libro: {metadata_orig['libro']}")
        print(f"Capítulo: {metadata_orig['capitulo']}")
        print(f"Tema principal: {metadata_orig['tema_principal']}")
        print(f"Personaje principal: {metadata_orig['personaje_principal']}")
        print(f"Todos los temas: {metadata_orig['todos_temas']}")
        print(f"Texto: {textos[idx][:150]}...")
        
        print(f"\nTOP {top_k} DOCUMENTOS MÁS SIMILARES (con análisis temático):")
        
        for i, idx_sim in enumerate(indices_similares):
            metadata_sim = metadatos_enriquecidos[idx_sim]
            sim_score = similaridades[idx_sim]
            
            # Analizar coincidencias
            mismo_libro = metadata_sim['libro'] == metadata_orig['libro']
            mismo_tema = metadata_sim['tema_principal'] == metadata_orig['tema_principal']
            mismo_personaje = metadata_sim['personaje_principal'] == metadata_orig['personaje_principal']
            
            # Calcular temas en común
            temas_comunes = set(metadata_orig['todos_temas']) & set(metadata_sim['todos_temas'])
            personajes_comunes = set(metadata_orig['todos_personajes']) & set(metadata_sim['todos_personajes'])
            
            # Crear indicadores
            indicadores = []
            if mismo_libro:
                indicadores.append("📖 MISMO LIBRO")
            if mismo_tema:
                indicadores.append("🎭 MISMO TEMA")
            if mismo_personaje:
                indicadores.append("👤 MISMO PERSONAJE")
            if temas_comunes:
                indicadores.append(f"🔗 TEMAS COMUNES: {list(temas_comunes)}")
            if personajes_comunes:
                indicadores.append(f"👥 PERSONAJES COMUNES: {list(personajes_comunes)}")
            
            print(f"\n{i+1}. Similaridad: {sim_score:.4f}")
            print(f"   Libro: {metadata_sim['libro']}")
            print(f"   Tema: {metadata_sim['tema_principal']}")
            print(f"   Personaje: {metadata_sim['personaje_principal']}")
            if indicadores:
                print(f"   Coincidencias: {' | '.join(indicadores)}")
            print(f"   Texto: {textos[idx_sim][:100]}...")

# Ejecutar análisis enriquecido
analizar_similaridad_enriquecida(indices_seleccionados, X_stephen_king, textos_stephen_king, metadatos_enriquecidos)

In [14]:
if len(textos_clf) >= 10 and len(libros_validos) >= 2:
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(etiquetas_clf)
    
    X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
        textos_clf, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
    )
    
    print(f"División de datos:")
    print(f"Train: {len(X_train_clf)} capítulos")
    print(f"Test: {len(X_test_clf)} capítulos")
    
    vectorizers_to_test = {
        'tfidf_basic': TfidfVectorizer(max_features=3000),
        'tfidf_ngrams': TfidfVectorizer(ngram_range=(1, 2), max_features=4000),
        'tfidf_filtered': TfidfVectorizer(min_df=3, max_df=0.7, max_features=3000),
        'count_basic': CountVectorizer(max_features=3000),
        'count_ngrams': CountVectorizer(ngram_range=(1, 2), max_features=4000)
    }
    
    modelos_to_test = {
        'MultinomialNB': MultinomialNB(),
        'ComplementNB': ComplementNB()
    }
    
    param_grids = {
        'MultinomialNB': {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]},
        'ComplementNB': {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
    }
    
    mejores_resultados = []
    
    print(f"\nOptimizando modelos para clasificar libros de Stephen King...")
    
    for vec_name, vectorizer in vectorizers_to_test.items():
        print(f"\nProbando vectorizador: {vec_name}")
        
        # vectorizar
        X_train_vec = vectorizer.fit_transform(X_train_clf)
        X_test_vec = vectorizer.transform(X_test_clf)
        
        for model_name, modelo in modelos_to_test.items():
            # grid search
            grid_search = GridSearchCV(
                modelo, param_grids[model_name], 
                cv=min(5, len(libros_validos)), 
                scoring='f1_macro',
                n_jobs=-1
            )
            
            grid_search.fit(X_train_vec, y_train_clf)
            
            # predecir en test
            y_pred = grid_search.predict(X_test_vec)
            f1_macro = f1_score(y_test_clf, y_pred, average='macro')
            
            mejores_resultados.append({
                'vectorizer': vec_name,
                'model': model_name,
                'best_params': grid_search.best_params_,
                'f1_macro': f1_macro,
                'grid_search': grid_search
            })
            
            print(f"  {model_name}: F1-macro = {f1_macro:.4f}, params = {grid_search.best_params_}")
    
else:
    print("No hay suficientes datos para entrenar modelos de clasificación.")
    print("Se necesitan al menos 10 capítulos y 2 libros con mínimo 3 capítulos cada uno.")

División de datos:
Train: 25 capítulos
Test: 11 capítulos

Optimizando modelos para clasificar libros de Stephen King...

Probando vectorizador: tfidf_basic


/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


  MultinomialNB: F1-macro = 0.2691, params = {'alpha': 0.1}
  ComplementNB: F1-macro = 0.6476, params = {'alpha': 0.1}

Probando vectorizador: tfidf_ngrams


/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated cla

  MultinomialNB: F1-macro = 0.2533, params = {'alpha': 0.1}
  ComplementNB: F1-macro = 0.7333, params = {'alpha': 0.1}

Probando vectorizador: tfidf_filtered


/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


  MultinomialNB: F1-macro = 1.0000, params = {'alpha': 0.1}
  ComplementNB: F1-macro = 1.0000, params = {'alpha': 0.1}

Probando vectorizador: count_basic


/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


  MultinomialNB: F1-macro = 0.8933, params = {'alpha': 0.1}
  ComplementNB: F1-macro = 1.0000, params = {'alpha': 0.1}

Probando vectorizador: count_ngrams
  MultinomialNB: F1-macro = 0.8933, params = {'alpha': 0.5}
  ComplementNB: F1-macro = 1.0000, params = {'alpha': 0.1}
  MultinomialNB: F1-macro = 0.8933, params = {'alpha': 0.5}
  ComplementNB: F1-macro = 1.0000, params = {'alpha': 0.1}


/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/mnt/c/Users/paoc3/OneDrive/Documents/Especializacion en IA/Septimo bimestre/pln_1/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [15]:
if 'mejores_resultados' in locals() and mejores_resultados:
    mejores_resultados_sorted = sorted(mejores_resultados, key=lambda x: x['f1_macro'], reverse=True)
    
    print("\n" + "="*80)
    print("MEJORES RESULTADOS DE CLASIFICACIÓN - LIBROS DE STEPHEN KING")
    print("="*80)
    
    for i, resultado in enumerate(mejores_resultados_sorted[:5]):
        print(f"\n{i+1}. F1-macro: {resultado['f1_macro']:.4f}")
        print(f"   Vectorizador: {resultado['vectorizer']}")
        print(f"   Modelo: {resultado['model']}")
        print(f"   Parámetros: {resultado['best_params']}")
    
    # mejor modelo
    mejor_resultado = mejores_resultados_sorted[0]
    mejor_grid = mejor_resultado['grid_search']
    
    print(f"\n\nANÁLISIS DETALLADO DEL MEJOR MODELO:")
    print(f"Vectorizador: {mejor_resultado['vectorizer']}")
    print(f"Modelo: {mejor_resultado['model']}")
    print(f"F1-macro: {mejor_resultado['f1_macro']:.4f}")
    
    # reporte
    vectorizer_name = mejor_resultado['vectorizer']
    vectorizer_usado = vectorizers_to_test[vectorizer_name]
    X_train_mejor = vectorizer_usado.fit_transform(X_train_clf)
    X_test_mejor = vectorizer_usado.transform(X_test_clf)
    
    y_pred_mejor = mejor_grid.predict(X_test_mejor)
    
    print("\nReporte de clasificación por libro:")
    print(classification_report(y_test_clf, y_pred_mejor, 
                              target_names=label_encoder.classes_))
    
    print("\nAnálisis de errores de clasificación:")
    for i, (true_label, pred_label) in enumerate(zip(y_test_clf, y_pred_mejor)):
        if true_label != pred_label:
            libro_real = label_encoder.classes_[true_label]
            libro_pred = label_encoder.classes_[pred_label]
            print(f"  Error: '{libro_real}' clasificado como '{libro_pred}'")
            
else:
    print("No se pudieron entrenar modelos de clasificación.")


MEJORES RESULTADOS DE CLASIFICACIÓN - LIBROS DE STEPHEN KING

1. F1-macro: 1.0000
   Vectorizador: tfidf_filtered
   Modelo: MultinomialNB
   Parámetros: {'alpha': 0.1}

2. F1-macro: 1.0000
   Vectorizador: tfidf_filtered
   Modelo: ComplementNB
   Parámetros: {'alpha': 0.1}

3. F1-macro: 1.0000
   Vectorizador: count_basic
   Modelo: ComplementNB
   Parámetros: {'alpha': 0.1}

4. F1-macro: 1.0000
   Vectorizador: count_ngrams
   Modelo: ComplementNB
   Parámetros: {'alpha': 0.1}

5. F1-macro: 0.8933
   Vectorizador: count_basic
   Modelo: MultinomialNB
   Parámetros: {'alpha': 0.1}


ANÁLISIS DETALLADO DEL MEJOR MODELO:
Vectorizador: tfidf_filtered
Modelo: MultinomialNB
F1-macro: 1.0000

Reporte de clasificación por libro:
                                  precision    recall  f1-score   support

     El visitante - Stephen King       1.00      1.00      1.00         3
   Fin de guardia - Stephen King       1.00      1.00      1.00         2
            Holly - Stephen King       1.0

### 3: Análisis de similaridad entre palabras

In [16]:
X_transpuesta = X_stephen_king.T

print(f"Matriz transpuesta (término-documento):")
print(f"Forma: {X_transpuesta.shape}")
print(f"Cantidad de términos: {X_transpuesta.shape[0]}")
print(f"Cantidad de documentos (capítulos): {X_transpuesta.shape[1]}")

# indice -> palabra
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

print(f"\nEjemplos de palabras en el vocabulario de Stephen King:")
palabras_ejemplo = list(tfidfvect.vocabulary_.keys())[:15]
for palabra in palabras_ejemplo:
    idx = tfidfvect.vocabulary_[palabra]
    print(f"  '{palabra}' -> índice {idx}")

Matriz transpuesta (término-documento):
Forma: (5000, 36)
Cantidad de términos: 5000
Cantidad de documentos (capítulos): 36

Ejemplos de palabras en el vocabulario de Stephen King:
  'once' -> índice 3312
  'asesinado' -> índice 270
  'pruebas' -> índice 3740
  'flint' -> índice 1911
  'city' -> índice 606
  'terry' -> índice 4564
  'maitland' -> índice 2890
  'entrenador' -> índice 1691
  'liga' -> índice 2727
  'infantil' -> índice 2293
  'profesor' -> índice 3722
  'literatura' -> índice 2739
  'marido' -> índice 2927
  'ejemplar' -> índice 1378
  'niñas' -> índice 3174


In [17]:
palabras_interes = ['holly', 'detective', 'hodges', 'finders', 'keepers']
vocabulario = tfidfvect.vocabulary_

print(f"\nPalabras seleccionadas para análisis de similaridad:")
for i, palabra in enumerate(palabras_interes):
    if palabra in vocabulario:
        idx = vocabulario[palabra]
        freq = np.array(X_stephen_king.sum(axis=0))[0][idx]
        print(f"{i+1}. '{palabra}' (índice: {idx}, frecuencia total: {freq:.2f})")


Palabras seleccionadas para análisis de similaridad:
1. 'holly' (índice: 2203, frecuencia total: 6.52)
2. 'detective' (índice: 1240, frecuencia total: 0.12)
3. 'hodges' (índice: 2180, frecuencia total: 6.85)
4. 'finders' (índice: 1907, frecuencia total: 0.19)
5. 'keepers' (índice: 2411, frecuencia total: 0.19)


In [18]:
def analizar_similaridad_palabra_stephen_king(palabra, X_transpuesta, vocabulario, idx2word, top_k=5):
    """
    Analiza la similaridad de una palabra con todas las demás palabras.
    """
    if palabra not in vocabulario:
        print(f"La palabra '{palabra}' no está en el vocabulario.")
        return None, None
    
    idx_palabra = vocabulario[palabra]
    
    similaridades = cosine_similarity(X_transpuesta[idx_palabra], X_transpuesta)[0]
    
    indices_similares = np.argsort(similaridades)[::-1][1:top_k+1]
    
    print(f"\n{'='*70}")
    print(f"ANÁLISIS DE SIMILARIDAD - PALABRA: '{palabra}'")
    print(f"{'='*70}")
    
    frecuencia_original = np.array(X_stephen_king.sum(axis=0))[0][idx_palabra]
    print(f"\nPALABRA ORIGINAL: '{palabra}'")
    print(f"Frecuencia total: {frecuencia_original:.2f}")
    
    print(f"\nTOP {top_k} PALABRAS MÁS SIMILARES:")
    palabras_similares = []
    
    for i, idx_sim in enumerate(indices_similares):
        palabra_similar = idx2word[idx_sim]
        sim_score = similaridades[idx_sim]
        frecuencia_similar = np.array(X_stephen_king.sum(axis=0))[0][idx_sim]
        
        print(f"{i+1:2d}. '{palabra_similar}' (similaridad: {sim_score:.4f}, freq: {frecuencia_similar:.2f})")
        palabras_similares.append((palabra_similar, sim_score, frecuencia_similar))
    
    return indices_similares, palabras_similares

resultados_palabras = {}

print("Analizando similaridades entre palabras en los libros de Stephen King...")
for palabra in palabras_interes:
    indices, similares = analizar_similaridad_palabra_stephen_king(
        palabra, X_transpuesta, vocabulario, idx2word
    )
    if similares:
        resultados_palabras[palabra] = similares

Analizando similaridades entre palabras en los libros de Stephen King...

ANÁLISIS DE SIMILARIDAD - PALABRA: 'holly'

PALABRA ORIGINAL: 'holly'
Frecuencia total: 6.52

TOP 5 PALABRAS MÁS SIMILARES:
 1. 'holly se' (similaridad: 0.9408, freq: 0.63)
 2. 'holly lo' (similaridad: 0.9373, freq: 0.21)
 3. 'holly no' (similaridad: 0.9137, freq: 0.41)
 4. 'que holly' (similaridad: 0.9081, freq: 0.31)
 5. 'gibney' (similaridad: 0.8938, freq: 0.51)

ANÁLISIS DE SIMILARIDAD - PALABRA: 'detective'

PALABRA ORIGINAL: 'detective'
Frecuencia total: 0.12

TOP 5 PALABRAS MÁS SIMILARES:
 1. 'gripe' (similaridad: 0.7893, freq: 0.10)
 2. 'le importa' (similaridad: 0.7729, freq: 0.07)
 3. 'mensaje' (similaridad: 0.7656, freq: 0.41)
 4. 'ella le' (similaridad: 0.7626, freq: 0.21)
 5. 'madre se' (similaridad: 0.7524, freq: 0.08)

ANÁLISIS DE SIMILARIDAD - PALABRA: 'hodges'

PALABRA ORIGINAL: 'hodges'
Frecuencia total: 6.85

TOP 5 PALABRAS MÁS SIMILARES:
 1. 'hodges no' (similaridad: 0.9599, freq: 0.57)
 2. 'd